# Pipelines for inferenece

本チュートリアルでは、`pieline()` を用いて、訓練済みモデルによって様々な推論タスクを実行する手法を学びます。\
なお、ここで紹介されている多くのモデルは非常に沢山のパラメータをもつため、ロードのために多くのメモリやストレージを必要とします。\
ローカルで実行する際は注意してください。

本チュートリアルは、[Hugging Face Transformers チュートリアル](https://huggingface.co/docs/transformers/v4.57.0/ja/pipeline_tutorial) を元に、一部加筆・修正して作成しています。

## Dependencies

このチュートリアルコードをすべて実行するためには、明示的に `import` するライブラリの他に、以下のソフトウェアが必要です。

- [`ffmpeg`](https://www.ffmpeg.org/) (および、その Python ラッパー: `ffmpeg-python`): 動画処理
- [`tesseract`](https://github.com/tesseract-ocr/tesseract) (および、その Python ラッパー: `pytesseract`): 画像処理
- `accelerate` ライブラリ: モデルの自動配置
- `bitsandbytes` ライブラリ: モデルの量子化
    - linux, windows のみサポート
- `pillow` ライブラリ: 画像処理
- `torchcodec` ライブラリ: 動画処理

もし自分の環境にインストールされていない場合には、事前にインストールしておいてください。\
なお、`ffmpeg` と `tesseract` に関しては、macOSであれば [`Homebrew`](https://formulae.brew.sh) から簡単にインストールできるようです (動作未確認) 。

In [ ]:
# run this cell if you are working in google colab

!pip install accelerate bitsandbytes datasets ffmpeg-python transformers torch torchcodec pillow pytesseract

In [ ]:
# import dependencies

from datasets import load_dataset
import torch
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

## Pipeline usage

`pipeline(task="task")` により、推論タスク `"task"` を行うデフォルトのモデルが提供されます。\
提供される構造体は、事前処理・推論・事後処理をワンライナーで行います。

In [ ]:
# prepare generator with task name
# default model: "facebook/wav2vec2-base-960h" (94.4M params)
# ref: (https://huggingface.co/facebook/wav2vec2-base-960h)

generator = pipeline(task="automatic-speech-recognition")

ファイルを指定して、推論を行います。

<audio src="https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac" controls></audio>

In [ ]:
# generate with input text
# target: "I have a dream that one day this nation will rise up and live out the true meaning of its creed ..." (Martin Luther King Jr.)

generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

具体的なモデルを指定するには、`pipeline(model="model")` とします。\
モデルの一覧は [`Hub`](https://huggingface.co/models) から確認できます。

In [ ]:
# prepare generator with model name
# superior model: "openai/whisper-large" (1.54B params, heavy)

generator = pipeline(model="openai/whisper-large")
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

複数の入力を `list` で受け取ることもできます。

<audio src="https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac" controls></audio>
<audio src="https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac" controls></audio>

In [ ]:
# target 1: "I have a dream that one day this nation will rise up and live out the true meaning of its creed ..." (Martin Luther King Jr.)
# target 2: "He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flour-fattened sauce." (James Joyce, "A Portrait Of The Artist As A Young Man")

generator(
    [
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac",
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac",
    ]
)

## Parameter
`pipeline()` はタスク固有・非固有の多くのパラメータをサポートしています。\
一般的には、このパラメータはどこでも指定できます。

```python
generator = pipeline(model="openai/whisper-large", my_parameter=1)
out = generator(...)                  # `my_parameter=1` is used here
out = generator(..., my_parameter=2)  # `my_parameter=2` is used here (overwritten)
out = generator(...)                  # `my_parameter=1` is used again here
```

以下で、特に重要なパラメータを確認しましょう。

### Device

`device=n` を使用すると、モデルが指定したデバイスに自動的に配置されます。\
具体的な use case は以下の通りです。

- `device=-1`: CPU
- `device=n` (non-negative integer): id `n` に対応する GPU
    - id はマシン上の各 GPU に自動的に割り振られています
    - NVIDIA GPU であれば `torch.cuda.get_device_name(n)` で id `n` に対応する GPU の名前が取得できます

なお、特に指定しない場合には、GPU を使用するように自動的にデバイスが決定されるようです。\
筆者の環境 (Macbook Air M4) では、Apple GPU (`mps`) が自動的に設定されました。

In [ ]:
# run "openai/whisper-large" on GPU[0]

generator = pipeline(model="openai/whisper-large", device=0)

もしモデルが単一のGPUには大きすぎる場合、`device_map="auto"` を指定して、`🤗 Accelerate` にモデルの重みをどのようにロードし、保存するかを自動的に決定させることができます。

In [ ]:
# run "openai/whisper-large" on GPU (loaded)

generator = pipeline(model="openai/whisper-large", device_map="auto")

### Batch Size

`batch_size=n` を指定することで、バッチサイズ `n` で推論することができます。\
ただし、バッチ処理は必ずしも速いわけではなく、実際にいくつかのケースでかなり遅くなることが確認されているようです。\
なお、バッチ処理を行ったとしても、得られる結果はバッチ処理を行わない場合と一致します。

In [ ]:
# batched inference

generator = pipeline(model="openai/whisper-large", device=0, batch_size=2)
audio_filenames = [f"audio_{i}.flac" for i in range(10)]
texts = generator(audio_filenames)

### Task specific parameters

すべてのタスクはタスク固有のパラメータを提供しています。\
たとえば、`transformers.AutomaticSpeechRecognitionPipeline.call()` メソッドには、ビデオの字幕作成に有用な `return_timestamps` パラメータがあります。

In [ ]:
# speech recognition with time stamp
# model: "facebook/wav2vec2-large-960h-lv60-self" (317M params)
# ref: https://huggingface.co/facebook/wav2vec2-large-960h-lv60-self

generator = pipeline(model="facebook/wav2vec2-large-960h-lv60-self", return_timestamps="word")
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

## Using pipeline in a dataset

`pipeline()` は大規模なデータセット上で推論を実行することもできます。\
例えば、イテレータを用いて、これを簡単に実行できます。

In [ ]:
# speech recognition with time stamp
# model: "openai-community/gpt2" (137M params)
# ref: https://huggingface.co/openai-community/gpt2

def data():
    for i in range(1000):
        yield f"My example {i}"

pipe = pipeline(model="openai-community/gpt2", device=0)
generated_characters = 0
for out in pipe(data()):
    generated_characters += len(out[0]["generated_text"])

`🤗 Datasets` からデータセットをロードして繰り返し反復させることもできます。

In [ ]:
# KeyDataset is a util that will just output the item we're interested in.

pipe = pipeline(model="hf-internal-testing/tiny-random-wav2vec2", device=0)
dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation[:10]")

for out in pipe(KeyDataset(dataset, "audio")):
    print(out)

## Using pipelines for a webserver

skip this section

## Vision pipeline

画像処理タスクに `pipeline()` を用いる方法もほぼ同じです。\
ここでは、写真上のオブジェクトを分類する推論タスクを実行しています。

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg" width="30%">

In [ ]:
# model: "google/vit-base-patch16-224" (86.6M params)
# ref: https://huggingface.co/google/vit-base-patch16-224

vision_classifier = pipeline(model="google/vit-base-patch16-224")
preds = vision_classifier(
    inputs="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg",
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

## Text pipeline

上と同様です。\
ここでは、テキストのコンテンツを分類する推論タスクを実行しています。

In [ ]:
# model: "facebook/bart-large-mnli" (407M params)
# ref: https://huggingface.co/facebook/bart-large-mnli

classifier = pipeline(model="facebook/bart-large-mnli")
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

## Multimodal pipeline

`pipeline()` は、複数のモダリティをサポートしています。\
たとえば、テキスト処理と画像処理を組み合わせて、画像についてテキストベースの推論タスクを実行することができます。

<img src="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png" width="30%">

In [ ]:
# model: "impira/layoutlm-document-qa" (128M params)
# ref: https://huggingface.co/impira/layoutlm-document-qa

vqa = pipeline(model="impira/layoutlm-document-qa")
output = vqa(
    image="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png",
    question="What is the invoice number?",
)
output[0]["score"] = round(output[0]["score"], 3)
output

## Using pipeline on large models with 🤗 accelerate

`device_map="auto"` を指定して、大規模モデルを使用可能なデバイスに適切に分配してロードします。

In [ ]:
# model: "facebook/opt-1.3b" (1.3B params, heavy)
# ref: https://huggingface.co/facebook/opt-1.3b

pipe = pipeline(model="facebook/opt-1.3b", dtype=torch.bfloat16, device_map="auto")
pipe("これは素晴らしい例です！", do_sample=True, top_p=0.95)

さらに、`bitsandbytes` をインストールし、`load_in_8bit=True` を指定すれば、モデルを 8 bit で読み込むことができます。\
なお、`bitsandbytes` は現状 linux と windows しかサポートしていません。

In [ ]:

pipe = pipeline(model="facebook/opt-1.3b", device_map="auto", model_kwargs={"load_in_8bit": True})
pipe("This is a cool example!", do_sample=True, top_p=0.95)